In [ ]:
!git clone https://github.com/madi82/Face-Payment.git
%cd Face-Payment/
!pip install retina-face

Cloning into 'Face-Payment'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 89 (delta 27), reused 68 (delta 16), pack-reused 0
Unpacking objects: 100% (89/89), 4.91 MiB | 4.00 MiB/s, done.
/content/Face-Payment
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from liveliness import test
import ArcFace
from retinaface import RetinaFace
from retinaface.commons import postprocess
from PIL import Image as Img
import matplotlib.pyplot as plt
import numpy as np
import json
from os import path
import logging
from tensorflow.keras.preprocessing import image
import time
import random


# Global variable
#---------------------------------------
# Main path to store all photos of user
path_database = "/content/Face-Payment/database/"
path_logs = "/content/Face-Payment/logs/"
cosine_threshold = 0.68
trans_key = 0
loaded_db = []
logging.basicConfig(filename=path_logs+'app.log',
                    filemode='a',
                    format='%(asctime)s, %(levelname)s, %(message)s',
                    level=logging.DEBUG,
                    force=True)
model = ArcFace.loadModel()

# For Video stream
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
import html
from base64 import b64decode, b64encode
import cv2
import PIL
import io

def jsob_to_image(js_object):
  # decode base64 image
  image_bytes = b64decode(js_object.split(',')[1])
  # convert bytes to numpy array
  img_array = np.frombuffer(image_bytes, dtype=np.uint8)
  # convert numpy array into OpenCV BGR 
  frame = cv2.imdecode(img_array, flags=1)

  return frame


def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="blue: red; font-weight: bold;">' +
          'click here to stop the video</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; 
      captureCanvas.height = 480; 
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame() {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
  
            

      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame():
  data = eval_js('stream_frame()')
  return data


#-------------------------------------------------------------------
def detect_face(path_specific_image: np.ndarray):

  detected_face = []
  align_det_face = [] 
  facial_information = []

  facial_information = RetinaFace.detect_faces(path_specific_image)
  logging.info("Facial information: %s",facial_information)

  for k, v in facial_information.items():
    facial_area = v['facial_area']
    landmarks = v['landmarks']
    detected_face = path_specific_image[facial_area[1]: facial_area[3], facial_area[0]: facial_area[2]]
    logging.info("Check detected image of Transaction ID : %s",trans_key)
    cv2.imwrite(path_logs+"DetectedImage_"+str(trans_key)+".jpg", detected_face)
    
    # Below code is to represent square on face and landmarks
    # highlight_image = path_specific_image
    # cv2.rectangle(highlight_image, (facial_area[2], facial_area[3]), (facial_area[0], facial_area[1]), (255, 255, 255), 1)
    # cv2.imwrite(path_logs+"HighlightImage_"+str(trans_key)+".jpg", highlight_image)

    # landmark_image = path_specific_image
    # cv2.circle(landmark_image, (round(landmarks["left_eye"][0]),round(landmarks["left_eye"][1])), 5, (0, 0, 255), -1)
    # cv2.circle(landmark_image, (round(landmarks["right_eye"][0]),round(landmarks["right_eye"][1])), 5, (0, 0, 255), -1)
    # cv2.circle(landmark_image, (round(landmarks["nose"][0]),round(landmarks["nose"][1])), 5, (0, 0, 255), -1)
    # cv2.circle(landmark_image, (round(landmarks["mouth_left"][0]),round(landmarks["mouth_left"][1])), 5, (0, 0, 255), -1)
    # cv2.circle(landmark_image, (round(landmarks["mouth_right"][0]),round(landmarks["mouth_right"][1])), 5, (0, 0, 255), -1)
    # cv2.imwrite(path_logs+"LandmarkImage_"+str(trans_key)+".jpg", landmark_image)

  # Alignment of image
  left_eye = landmarks["left_eye"]
  right_eye = landmarks["right_eye"]
  nose = landmarks["nose"]
  align_det_face = postprocess.alignment_procedure(detected_face, right_eye, left_eye, nose)
  cv2.imwrite(path_logs+"AlingedImage_"+str(trans_key)+".jpg", align_det_face)
  
  return detected_face, align_det_face, facial_information

#-------------------------------------------------------------------
def preprocess_face(img, target_size=(112,112)):

    #resize image to expected shape
    if img.shape[0] > 0 and img.shape[1] > 0:
      factor_0 = target_size[0] / img.shape[0]
      factor_1 = target_size[1] / img.shape[1]
      factor = min(factor_0, factor_1)

      dsize = (int(img.shape[1] * factor), int(img.shape[0] * factor))
      img = cv2.resize(img, dsize)

      # Then pad the other side to the target size by adding black pixels
      diff_0 = target_size[0] - img.shape[0]
      diff_1 = target_size[1] - img.shape[1]
      img = np.pad(img, ((diff_0 // 2, diff_0 - diff_0 // 2), (diff_1 // 2, diff_1 - diff_1 // 2), (0, 0)), 'constant')

    logging.info("Check pre-processed image of Transaction ID : %s",trans_key)
    cv2.imwrite(path_logs+"PreprocessedImage_"+str(trans_key)+".jpg", img)

    #normalizing the image pixels
    img_pixels = image.img_to_array(img)
    img_pixels = np.expand_dims(img_pixels, axis = 0)
    img_pixels /= 255 #normalize input in [0, 1]


    # Reference study: The faces are cropped and resized to 112×112,
    # and each pixel (ranged between [0, 255]) in RGB images is normalised
    # by subtracting 127.5 then divided by 128.
    #img_pixels -= 127.5
    #img_pixels /= 128
    return img_pixels

#-------------------------------------------------------------------
def representation(img):
  return model.predict(img, verbose=0)[0].tolist()

#-------------------------------------------------------------------
def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

#-------------------------------------------------------------------
# This function used for Face recognition
def get_all_distance(target_embedding):
  distances = dict()
  for key in loaded_db:
    source_embedding = loaded_db[key]["embedding"]
    distance = findCosineDistance(source_embedding, target_embedding)
    distances[key] = distance

  minimum_keys = [key for key in distances if distances[key]==min(distances.values())]
  under_threshold_distance = dict()
  for key in distances:
    if distances[key] <= cosine_threshold:
      under_threshold_distance[key] = distances[key]

  predicted_person = [k for k, v in under_threshold_distance.items() if v == min(under_threshold_distance.values())]
  predicted_person = predicted_person[0]

  return predicted_person, distances

#-------------------------------------------------------------------
def load_database():
  # Read JSON file
  with open(path_database+"database.json") as fp:
    jsonload = json.load(fp)
  logging.info("Database loaded")
  return jsonload

loaded_db = load_database()
logging.info("Loaded DB : %s ", loaded_db)


def menu():
  print("\n---------------------------------------")
  logging.info("\n---------------------------------------")
  strs = ('\n\n'
          'Enter 1 for Payment by Face?\n'
          'Enter 2 for Create account?\n'
          'Enter 5 to exit : ')
  choice = input(strs)
  return int(choice) 

while True:          
  choice = menu()
  if choice == 1:
    print("\n\n---------------------------------------")
    trans_key = random.randint(1000, 9999)
    logging.info("Transaction ID : %s", trans_key)
    # Get the webcam stream and forward it to python 
    video_stream()
    validation = 1
    while True:
      frame_js = video_frame()
      if not frame_js:
        break
      img = jsob_to_image(frame_js["img"])
      if validation == 1:
        validation = validation - 1
        logging.info("Check raw image of Transaction ID : %s",trans_key)
        cv2.imwrite(path_logs+"RawImage_"+str(trans_key)+".jpg", img)
        logging.info("Checking for Anti-Spoofing")
        anti_spoof_res = test(img,'/content/Face-Payment/resources/anti_spoof_models', 0)
        logging.info("Real/Fake face label: %s and Pred acc: %s and Pred time: %s",anti_spoof_res[0],anti_spoof_res[1],anti_spoof_res[2])
        if anti_spoof_res[0] == 1:
          starttime = time.time()
          print(f"Transaction in progress...")
          predname = get_all_distance(representation(preprocess_face(detect_face(img)[1])))
          logging.info("Predicted name: %s and All Distances: %s",predname[0],predname[1])
          cust_name = predname[0]                
          cust_bal = loaded_db[cust_name]["balance"]
          print(f"Welcome {cust_name}")
          print(f"Account balance : {cust_bal} CAD")
          updated_cust_bal = int(cust_bal)-20
          print(f"Remaining account balance : {updated_cust_bal} CAD")
          loaded_db[cust_name]["balance"] = updated_cust_bal
          logging.info("Prediction cost: %s",round((time.time() - starttime), 2))
        else:
          print(f"Access denied!!!")
  elif choice == 2:
    print("\n\n---------------------------------------")
    trans_key = random.randint(1000, 9999)
    logging.info("Transaction ID : %s", trans_key)
    # Get the webcam stream and forward it to python 
    video_stream()
    validation = 1
    while True:
      frame_js = video_frame()
      if not frame_js:
        break
      img = jsob_to_image(frame_js["img"])
      if validation == 1:
        validation = validation - 1
        logging.info("Check raw image of Transaction ID : %s",trans_key)
        label = input("Enter your name: ")
        balance = input("Enter your balance: ")
        cv2.imwrite(path_database+""+label+".jpg", img)
        embedding = representation(preprocess_face(detect_face(img)[1]))
        loaded_db[label] = {"embedding": embedding,"balance": balance}
        print(f"User enrolled!!!")
  elif choice == 5:
    break

arcface_weights.h5  will be downloaded to  /root/.deepface/weights/arcface_weights.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: /root/.deepface/weights/arcface_weights.h5
100%|██████████| 137M/137M [00:01<00:00, 72.7MB/s]



---------------------------------------


Enter 1 for Payment by Face?
Enter 2 for Create account?
Enter 5 to exit : 2


---------------------------------------


<IPython.core.display.Javascript object>

Enter your name: Shantanu
Enter your balance: 890
retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5
100%|██████████| 119M/119M [00:01<00:00, 95.1MB/s]


User enrolled!!!

---------------------------------------


Enter 1 for Payment by Face?
Enter 2 for Create account?
Enter 5 to exit : 5
